In [1]:
%load_ext dotenv
%dotenv

In [6]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    branch="master",
    repo_path="./langchain",
    file_filter=file_filter
)

documents = loader.load()
print(len(documents))

439


In [13]:
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings

emb = AzureOpenAIEmbeddings(
    model = "text-embedding-ada-002"
)
# AOAI の RateLimit で 429 を避けるため、ドキュメントの数を絞る
db = Chroma.from_documents(documents[:5], emb)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI

prompt = ChatPromptTemplate.from_template(
    '''
    以下の文脈だけを参考にして、質問に答えてください。
    文脈: {context}
    質問: {question}
    '''
)

In [24]:
import os
from azure.identity import ClientSecretCredential, get_bearer_token_provider

token_provider = get_bearer_token_provider(
    ClientSecretCredential(
        client_id=os.getenv("client_id"),
        client_secret=os.getenv("client_secret"),
        tenant_id=os.getenv("tenant_id")
    ),
    "https://cognitiveservices.azure.com/.default"
)
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-02-15-preview",
    azure_ad_token_provider=token_provider,
    temperature=0
)

In [25]:
retriever = db.as_retriever()

In [ ]:
chain = {
    "context": retriever,
    "question": RunnablePassthrough()
} | prompt | llm | StrOutputParser()

output = chain.invoke("LangChainについて教えてください。")
print(output)

In [33]:
chain = {
    "context": retriever,
    "question": RunnablePassthrough()
} | RunnablePassthrough.assign( ans = prompt | llm | StrOutputParser())

output = chain.invoke("LangChainについて教えてください。")
print(output['ans'])

LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMを活用したアプリケーションのライフサイクル全体を簡素化することを目的としています。以下にLangChainの主な特徴と機能を説明します。

---

### **主な特徴**
1. **開発（Development）**:
   - LangChainは、オープンソースの[コンポーネント](/docs/concepts)や[サードパーティ統合](/docs/integrations/providers/)を利用してアプリケーションを構築することを支援します。
   - [LangGraph](/docs/concepts/architecture/#langgraph)を使用して、ストリーミングや人間の介入をサポートする状態管理エージェントを構築できます。

2. **プロダクション化（Productionization）**:
   - [LangSmith](https://docs.smith.langchain.com/)を使用して、アプリケーションの検査、監視、評価を行い、継続的な最適化と信頼性のあるデプロイを実現します。

3. **デプロイメント（Deployment）**:
   - [LangGraph Platform](https://docs.langchain.com/langgraph-platform)を利用して、LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換できます。

---

### **アーキテクチャ**
LangChainは複数のオープンソースライブラリで構成されています：
- **`langchain-core`**: チャットモデルやその他のコンポーネントの基本的な抽象化を提供。
- **統合パッケージ**: 重要な統合（例: `langchain-openai`, `langchain-anthropic`）は軽量パッケージとして分割され、LangChainチームと統合開発者によって共同管理されています。
- **`langchain`**: アプリケーションの認知アーキテクチャを構成するチェーン、エージェント、検索戦略を提供。
- **`lang